In [2]:
!pip install openai
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.0 MB/s eta 0:00:00


In [3]:
import json
import pprint
import openai
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
def create_prompt_query(topic):
    query = f'''
          Create 1 code completion exercise for “{topic}” in JavaScript.

          The exercise should follow the pattern:

          ```
          def name(args):

          javascript code to solve the exercise
   
          MAKE IT FOR BEGINNERS IN JAVASCRIPT

          GIVE ONLY THE CODE FOR THE EXERCISE.

          DON'T MAKE CLASSES

          '''

    query = "\n".join([m.lstrip() for m in query.strip().split("\n")])

    return query


In [7]:
def create_prompts():
    topics = ["Sorting", "Binary Search Trees", "Linked Lists"]
    prompts = []

    for topic in topics:
        query = create_prompt_query(topic)
        prompts.append(query)

    return prompts

In [8]:
def generate_prompt_util(prompt_template):
    try:
        prompt = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_template}],
            max_tokens=250,
            timeout=60
            )
        return prompt
    except Exception as e:
        print(f"Error: {e}")
        return None

In [9]:
def generate_prompt():
    prompts_dict = {}
    query_num = 1
    
    for i in range(3):
        for prompt_template in prompts:
            response = generate_prompt_util(prompt_template)
            prompt = response.choices[0].message.content
            prompts_dict["Query_" + str(query_num)] = prompt
            query_num += 1
    
    prompts_json = json.dumps(prompts_dict)
    
    return prompts_json

In [10]:
prompts = create_prompts()

In [11]:
prompts_from_api = generate_prompt()

In [12]:
pprint.pprint(prompts_from_api)

('{"Query_1": "```javascript\\n// Exercise: Sort an array of numbers in '
 'ascending order\\n\\n// Input: array of numbers\\n// Output: sorted array of '
 'numbers\\n\\n// Example:\\n// Input: [5, 2, 9, 1, 3]\\n// Output: [1, 2, 3, '
 '5, 9]\\n\\n// Code:\\nconst numbers = [5, 2, 9, 1, '
 '3];\\nnumbers.sort(function(a, b) {\\n  return a - '
 'b;\\n});\\n\\nconsole.log(numbers);\\n```\\n\\nInstructions for the '
 'exercise:\\n\\nWrite code to sort the given array of numbers in ascending '
 'order. Fill in the missing part of the code to achieve the desired output. '
 'The sorted array should be printed using the `console.log()` '
 'function.\\n\\nNote: The given code already declares an array of numbers and '
 'provides a function as an argument to the `sort()` method. Your task is to '
 'complete the function using the correct parameters to ensure the array is '
 'sorted correctly.\\n\\nTest your code by running it and see if the output '
 'matches the expected sorted array.", "Query

In [13]:
model = AutoModelForCausalLM.from_pretrained("jinaai/starcoder-1b-textbook", device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("jinaai/starcoder-1b-textbook")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
prompt = '''
generate a javascript function that performs a bubble sort algorithm given
a list of items
'''

inputs = tokenizer(prompt.rstrip(), return_tensors="pt").to("cuda")

generation_output = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
)

s = generation_output.sequences[0]
output = tokenizer.decode(s, skip_special_tokens=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



generate a javascript function that performs a bubble sort algorithm given
a list of items in the list.


# Example usage
items = [1, 2, 3, 4, 5]
print(sort_items(items))
# Output: [1, 2, 3, 4, 5]

# Exercise:
# Complete the code below to implement the bubble sort algorithm
# using a for loop and the range function.
# The algorithm should swap adjacent elements if they are in the wrong order.
# The algorithm should stop when the list is sorted.


# Solution:
for i in range(len(items) - 1):
    for j in range(len(items) - 1 - i):
        if items[j] > items[j + 1]:
            items[j], items[j + 1] = items[j + 1], items[j]

print(items)
# Output: [5, 4, 3, 2, 1]

# Exercise:
# Complete the code below to implement the bubble sort algorithm
# using a for loop and the range function.
# The algorithm should swap adjacent elements if they are in the wrong order.
# The
